In [1]:
import quimb as qu
import joblib
import numpy as np

In [2]:
# define number of sites == number of qubits (each site is a 2-level system)
n = 4

In [3]:
def h_single_site(n, k):
    dims = (2,) * n
    def gen_term(i):
        return qu.ikron(qu.pauli(k), dims, [i])
    return sum(map(gen_term, range(0, n)))    

def h_two_sites(n, k):
    dims = (2,) * n
    def gen_term(i):
        return qu.ikron(qu.pauli(k), dims, [i, i+1])
    return sum(map(gen_term, range(0, n-1)))

def h_two_sites_j2(n, k):
    dims = (2,) * n
    def gen_term(i):
        return  qu.ikron(qu.pauli(k), dims, [i, i+2])
    return sum(map(gen_term, range(0, n-2)))

Label 0 (1D transversefield Ising model):
- $H = \sum_{n=1}^{N-1} Z_n Z_{n+1} + 2 \sum_{n=1}^{N} X_n$

Label 1 (1D Heisenberg model):
- $H = \sum_{n=1}^{N-1} (X_n X_{n+1} +  Y_n Y_{n+1} + Z_n Z_{n+1}) + 2 \sum_{n=1}^{N} Z_n$  

Label 3 (J1-J2 model):
- $H = \sum_{n=1}^{N-1} (X_n X_{n+1} +  Y_n Y_{n+1} + Z_n Z_{n+1}) + 3 \sum_{n=1}^{N-2} (X_n X_{n+2} +  Y_n Y_{n+2} + Z_n Z_{n+2}) $

In [4]:
def Hamiltonian_terms(n, label):
    h_terms = []
    
    if label == 0:
        h_terms.append(h_two_sites(n, "Z"))
        h_terms.append(h_single_site(n, "X"))
    if label == 1:
        h_terms.append(h_two_sites(n, "X") + h_two_sites(n, "Y") + h_two_sites(n, "Z"))
        h_terms.append(h_single_site(n, "Z"))
    if label == 2:
        pass  # to be implemented
    if label == 3:
        h_terms.append(h_two_sites(n, "X") + h_two_sites(n, "Y") + h_two_sites(n, "Z"))
        h_terms.append(h_two_sites_j2(n, "X") + h_two_sites_j2(n, "Y") + h_two_sites_j2(n, "Z"))
    if label == 4:
        pass  # to be implemented
    if label == 5:
        pass  # to be implemented

    return h_terms

It is defined as a function that returns a list of Hamiltonian terms so that it can be weighted by any coefficient you like.  
Note that the Hamiltonian is defined by the Pauli operator, not the spin operator.

## Sanity Check

### Label 0 (1D transversefield Ising model)

In [5]:
h_terms = Hamiltonian_terms(n, label=0)
h = h_terms[0] + 2*h_terms[1]  # same coefficients as the paper

eigen_values, eigen_vectors = qu.eigh(h)
print(eigen_values[0])
print(eigen_vectors[:, [0]].round(5))

-8.37679863685036
[[ 0.16887+0.j]
 [-0.21149+0.j]
 [-0.26883+0.j]
 [ 0.20489+0.j]
 [-0.26883+0.j]
 [ 0.35732+0.j]
 [ 0.26045+0.j]
 [-0.21149+0.j]
 [-0.21149+0.j]
 [ 0.26045+0.j]
 [ 0.35732+0.j]
 [-0.26883+0.j]
 [ 0.20489+0.j]
 [-0.26883+0.j]
 [-0.21149+0.j]
 [ 0.16887+0.j]]


In [6]:
ref = joblib.load(f"../VQE-generated-dataset/data/ground_state/{str(n).zfill(2)}qubit/label0.jb")
print(ref["ground_energy"])
print(ref["ground_state"].round(5).reshape(-1, 1))

-8.376798636850356
[[-0.03704-0.16476j]
 [ 0.04639+0.20634j]
 [ 0.05897+0.26228j]
 [-0.04494-0.1999j ]
 [ 0.05897+0.26228j]
 [-0.07838-0.34862j]
 [-0.05713-0.2541j ]
 [ 0.04639+0.20634j]
 [ 0.04639+0.20634j]
 [-0.05713-0.2541j ]
 [-0.07838-0.34862j]
 [ 0.05897+0.26228j]
 [-0.04494-0.1999j ]
 [ 0.05897+0.26228j]
 [ 0.04639+0.20634j]
 [-0.03704-0.16476j]]


In [7]:
qu.fidelity(eigen_vectors[:, 0], ref["ground_state"])

1.0000000000000002

### Label 1 (1D Heisenberg model)

In [8]:
h_terms = Hamiltonian_terms(n, label=1)
h = h_terms[0] + 2*h_terms[1]  # same coefficients as the paper

eigen_values, eigen_vectors = qu.eigh(h)
print(eigen_values[0])
print(eigen_vectors[:, [0]].round(5))

-7.828427124746192
[[ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.2706 +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.65328+0.j]
 [ 0.     +0.j]
 [-0.65328+0.j]
 [ 0.2706 +0.j]
 [ 0.     +0.j]]


In [9]:
ref = joblib.load(f"../VQE-generated-dataset/data/ground_state/{str(n).zfill(2)}qubit/label1.jb")
print(ref["ground_energy"])
print(ref["ground_state"].round(5).reshape(-1, 1))

-7.828427124746191
[[ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.2706 +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.65328+0.j]
 [ 0.     +0.j]
 [-0.65328+0.j]
 [ 0.2706 +0.j]
 [ 0.     +0.j]]


In [10]:
qu.fidelity(eigen_vectors[:, 0], ref["ground_state"])

1.0000000000000004

### Label 3 (J1-J2 model)

In [11]:
h_terms = Hamiltonian_terms(n, label=3)
h = h_terms[0] + 3*h_terms[1]  # same coefficients as the paper

eigen_values, eigen_vectors = qu.eigh(h)
print(eigen_values[0])
print(eigen_vectors[:, [0]].round(5))

-18.16515138991169
[[ 0.     +0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [ 0.47034+0.j]
 [ 0.     +0.j]
 [ 0.0548 +0.j]
 [-0.52514+0.j]
 [-0.     +0.j]
 [ 0.     +0.j]
 [-0.52514+0.j]
 [ 0.0548 +0.j]
 [-0.     +0.j]
 [ 0.47034+0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]]


In [12]:
ref = joblib.load(f"../VQE-generated-dataset/data/ground_state/{str(n).zfill(2)}qubit/label3.jb")
print(ref["ground_energy"])
print(ref["ground_state"].round(5).reshape(-1, 1))

-18.165151389911674
[[ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.     +0.j]
 [ 0.47034+0.j]
 [-0.     +0.j]
 [ 0.0548 +0.j]
 [-0.52514+0.j]
 [ 0.     +0.j]
 [-0.     +0.j]
 [-0.52514+0.j]
 [ 0.0548 +0.j]
 [-0.     +0.j]
 [ 0.47034+0.j]
 [-0.     +0.j]
 [-0.     +0.j]
 [ 0.     +0.j]]


In [13]:
qu.fidelity(eigen_vectors[:, 0], ref["ground_state"])

1.0